## Deliverable 2. Create a Customer Travel Destinations Map.

In [139]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [140]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Port Alfred,ZA,-33.5906,26.8910,57.18,95,100,3.00,overcast clouds
1,1,Hammerfest,NO,70.6634,23.6821,43.25,100,75,17.27,broken clouds
2,2,Namibe,AO,-15.1961,12.1522,63.46,84,89,9.75,overcast clouds
3,3,Barinas,VE,8.6226,-70.2075,87.46,57,96,6.44,overcast clouds
4,4,Lorengau,PG,-2.0226,147.2712,80.01,78,68,3.29,broken clouds


In [141]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 10
What is the maximum temperature you would like for your trip? 45


In [142]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hammerfest,NO,70.6634,23.6821,43.25,100,75,17.27,broken clouds
18,18,Punta Arenas,CL,-53.1500,-70.9167,43.29,59,4,7.85,clear sky
20,20,Te Anau,NZ,-45.4167,167.7167,43.95,96,100,2.17,overcast clouds
36,36,Qaanaaq,GL,77.4840,-69.3632,24.87,56,0,7.94,clear sky
37,37,Upernavik,GL,72.7868,-56.1549,37.83,74,99,5.19,overcast clouds
53,53,Barrow,US,71.2906,-156.7887,38.39,87,100,16.42,overcast clouds
56,56,General Roca,AR,-39.0333,-67.5833,44.62,64,100,17.87,overcast clouds
64,64,Ushuaia,AR,-54.8000,-68.3000,44.26,65,0,10.36,clear sky
80,80,Dubbo,AU,-32.2500,148.6167,41.13,100,99,0.00,overcast clouds
82,82,Pevek,RU,69.7008,170.3133,32.63,97,100,18.16,light snow


In [143]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                71
City                   71
Country                71
Lat                    71
Lng                    71
Max Temp               71
Humidity               71
Cloudiness             71
Wind Speed             71
Current Description    71
dtype: int64

In [144]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(inplace=True) 


C:\Users\baseb\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [145]:
preferred_cities_df.count()

City_ID                71
City                   71
Country                71
Lat                    71
Lng                    71
Max Temp               71
Humidity               71
Cloudiness             71
Wind Speed             71
Current Description    71
dtype: int64

In [146]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
518,Sangar,RU,28.81,few clouds,63.9242,127.4739,
531,Ust-Maya,RU,31.26,snow,60.4167,134.5333,
541,Launceston,AU,44.15,fog,-41.4500,147.1667,
563,Belyy Yar,RU,37.29,clear sky,53.6039,91.3903,
573,Arman,RU,42.67,overcast clouds,59.7000,150.1667,
575,Omsukchan,RU,32.63,broken clouds,62.5333,155.8000,
584,Dudinka,RU,39.87,overcast clouds,69.4058,86.1778,
598,Churapcha,RU,28.63,overcast clouds,62.0025,132.4325,
628,Talaya,RU,37.31,overcast clouds,61.3833,152.7500,
671,Paamiut,GL,43.21,overcast clouds,61.9940,-49.6678,


In [147]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [148]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.mask(hotel_df == '')
clean_hotel_df.dropna(inplace=True)
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hammerfest,NO,43.25,broken clouds,70.6634,23.6821,Skytterhuset
18,Punta Arenas,CL,43.29,clear sky,-53.1500,-70.9167,Hotel Hain
20,Te Anau,NZ,43.95,overcast clouds,-45.4167,167.7167,Kingsgate Hotel Te Anau
36,Qaanaaq,GL,24.87,clear sky,77.4840,-69.3632,Qaanaaq Hotel
37,Upernavik,GL,37.83,overcast clouds,72.7868,-56.1549,Café de Upernavik
53,Barrow,US,38.39,overcast clouds,71.2906,-156.7887,King Eider Inn
56,General Roca,AR,44.62,overcast clouds,-39.0333,-67.5833,Hotel El Recreo
64,Ushuaia,AR,44.26,clear sky,-54.8000,-68.3000,Albatross Hotel
80,Dubbo,AU,41.13,overcast clouds,-32.2500,148.6167,Quality Inn Dubbo International
82,Pevek,RU,32.63,light snow,69.7008,170.3133,Severnoye Zoloto


In [149]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [152]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description	</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [153]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))